In [1]:
import sys
sys.path.append("../")

In [7]:
import polars as pl
import yaml
from itertools import combinations

from utils.load_data import *

### Load data

In [3]:
with open("../config.yml", "r") as file:
    config = yaml.safe_load(file)

In [6]:
monoprix_gold = gold('../data/raw/', 'monoprix', config).collect()
franprix_gold = gold('../data/raw/', 'franprix', config).collect()
auchan_gold = gold('../data/raw/', 'auchan', config).collect()

In [8]:
datasets = [monoprix_gold, franprix_gold, auchan_gold]
pairs = list(combinations(datasets, 2))

l=[]
for pair in pairs:
    l.append (
        pl.concat(
            [
                pair[0].select(pl.col("product_id"), pl.col("brand_desc_slug").alias('brand_desc_left')),
                pair[1].select(pl.col("product_id"), pl.col("brand_desc_slug").alias('brand_desc_right')) 
            ], 
            how="align"
            )
            .filter(pl.col('brand_desc_left').is_not_null())
            .filter(pl.col('brand_desc_right').is_not_null())
            .filter(pl.col('brand_desc_left') != pl.col('brand_desc_right'))
            .groupby('brand_desc_left', 'brand_desc_right')
            .count()
            .filter(pl.col('count')>1)
            .select('brand_desc_left', 'brand_desc_right')
        )

In [10]:
dataset = (
    pl.concat(
        l, 
        how="vertical"
        )
)


# training_dataset_invert
dataset_invert = (
    dataset
    .with_columns(pl.col('brand_desc_left').alias('tmp'))
    .with_columns(pl.col('brand_desc_right').alias('brand_desc_left'))
    .with_columns(pl.col('tmp').alias('brand_desc_right'))
    .select(pl.col('brand_desc_left'), pl.col('brand_desc_right'))
)

result = (
    pl.concat(
        [
            dataset, 
            dataset_invert
        ], 
        how="vertical"
        )
        .unique()        
    .select(pl.col('brand_desc_left').alias('left_side'), pl.col('brand_desc_right').alias('right_side'))
)
result.shape

result.write_csv('../temp_folder/similar_brands_base_on_products.csv', separator=";")